In [3]:
import json
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pandas import json_normalize
from pymongo import MongoClient, errors
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import pos_tag
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import os
from pathlib import Path
from string import punctuation
from bs4 import BeautifulSoup
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
from gensim import corpora
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import guidedlda
import seaborn as sns
from sklearn.decomposition import LatentDirichletAllocation as LDA

In [7]:
df = pd.read_json('../data/concatenated_abridged.jsonl', lines = True)

In [8]:
df.head()

,created_at,id,id_str,full_text,truncated,display_text_range,entities,source,in_reply_to_status_id,in_reply_to_status_id_str,...,quoted_status,retweet_count,favorite_count,favorited,retweeted,possibly_sensitive,lang,retweeted_status,extended_entities,withheld_in_countries
0,2020-09-29 23:59:56+00:00,1311093385688735744,1311093385688735744,Biden a professional debater?? Hardly!! More ...,False,"[0, 71]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,NaN,...,{'created_at': 'Tue Sep 29 23:12:13 +0000 2020...,0,0,False,False,0.0,en,NaN,NaN,NaN
1,2020-09-29 23:59:58+00:00,1311093394400333824,1311093394400333824,RT @DarDar458: #PresidentialDebate #Debates202...,False,"[0, 73]","{'hashtags': [{'text': 'PresidentialDebate', '...","<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,...,NaN,1,0,False,False,NaN,und,{'created_at': 'Tue Sep 29 23:55:42 +0000 2020...,NaN,NaN
2,2020-09-30 00:00:00+00:00,1311093401652277252,1311093401652277248,How can Biden stand a chance against Trump in ...,False,"[0, 280]","{'hashtags': [{'text': 'VarneyCo', 'indices': ...","<a href=""https://studio.twitter.com"" rel=""nofo...",NaN,NaN,...,NaN,25,100,False,False,0.0,en,NaN,"{'media': [{'id': 1310969778111217665, 'id_str...",NaN
3,2020-09-30 00:00:02+00:00,1311093410737070086,1311093410737070080,RT @abbydphillip: Very reminiscent of pre-2018...,False,"[0, 108]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,...,NaN,351,0,False,False,NaN,en,{'created_at': 'Tue Sep 29 23:00:28 +0000 2020...,NaN,NaN
4,2020-09-30 00:00:04+00:00,1311093419100561408,1311093419100561408,RT @RudyGiuliani: Why was Corrupt Biden’s son ...,False,"[0, 110]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,...,NaN,15672,0,False,False,NaN,en,{'created_at': 'Tue Sep 29 19:07:59 +0000 2020...,NaN,NaN


In [6]:
text_df = df["full_text"]

In [7]:
text_df.head()

0    Biden a professional debater??  Hardly!! More ...
1    RT @DarDar458: #PresidentialDebate #Debates202...
2    How can Biden stand a chance against Trump in ...
3    RT @abbydphillip: Very reminiscent of pre-2018...
4    RT @RudyGiuliani: Why was Corrupt Biden’s son ...
Name: full_text, dtype: object

In [9]:
def remove_punctuation(string, punctuation):
    # remove given punctuation marks from a string
    for character in punctuation:
        string = string.replace(character, '')
    return string


def lemmatize_str(string):
    # Lemmatize a string and return it in its original format
    w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
    lemmatizer = nltk.stem.WordNetLemmatizer()
    return " ".join([lemmatizer.lemmatize(w)
                    for w in w_tokenizer.tokenize(string)
                    if "http" not in w])


def clean_column(df, column):
    # Apply data cleaning pipeline to a given pandas DataFrame column
    df[column] = df[column].apply(lambda x: str(x).lower())
    df[column] = df[column].apply(lambda x: remove_punctuation(x, punctuation))
    df[column] = df[column].apply(lambda x: lemmatize_str(x))
    return

In [5]:
def plot_10_most_common_words(count_data, count_vectorizer):
    words = count_vectorizer.get_feature_names()
    total_counts = np.zeros(len(words))
    for t in count_data:
        total_counts += t.toarray()[0]
    count_dict = (zip(words, total_counts))
    count_dict = sorted(count_dict, key=lambda x: x[1], reverse=True)[0:10]
    words = [w[0] for w in count_dict]
    counts = [w[1] for w in count_dict]
    x_pos = np.arange(len(words))
    plt.figure(2, figsize=(15, 15/1.6180))
    plt.subplot(title='10 most common words')
    sns.set_context("notebook", font_scale=1.25, rc={"lines.linewidth": 2.5})
    sns.barplot(x_pos, counts, palette='husl')
    plt.xticks(x_pos, words, rotation=90)
    plt.xlabel('words')
    plt.ylabel('counts')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig("../images/common_words.png")
    # plt.show()

In [11]:
clean_column(df, 'full_text')
print(df.head())

                 created_at                   id               id_str  \
0 2020-09-29 23:59:56+00:00  1311093385688735744  1311093385688735744   
1 2020-09-29 23:59:58+00:00  1311093394400333824  1311093394400333824   
2 2020-09-30 00:00:00+00:00  1311093401652277252  1311093401652277248   
3 2020-09-30 00:00:02+00:00  1311093410737070086  1311093410737070080   
4 2020-09-30 00:00:04+00:00  1311093419100561408  1311093419100561408   

                                           full_text  truncated  \
0  biden a professional debater hardly more like ...      False   
1  rt dardar458 presidentialdebate debates2020 tr...      False   
2  how can biden stand a chance against trump in ...      False   
3  rt abbydphillip very reminiscent of pre2018 el...      False   
4  rt rudygiuliani why wa corrupt biden’s son bei...      False   

  display_text_range                                           entities  \
0            [0, 71]  {'hashtags': [], 'symbols': [], 'user_mentions...   
1       